# GRU实现


## 1. 直接使用GRU，根据5种类型的数据，预测气温


读取数据，划分训练集和测试集


In [1]:
import pandas as pd
import torch
from Network import *

# 选取需要的列
COLUMN_INDEX = [
    "Temp (°C)",
    "Dew Point Temp (°C)",
    "Rel Hum (%)",
    "Wind Spd (km/h)",
    "Stn Press (kPa)",
]
data = pd.read_csv("Data/DataSet.csv")[COLUMN_INDEX].to_numpy()
data_tensor = torch.tensor(data, dtype=torch.float32)


# 设置随机种子以确保结果可重复
torch.manual_seed(42)
train_loader, test_loader = Split2Loaders(
    data_tensor, data_tensor, BATCHSIZE=32, RATIO=0.7, SHUFFLE=True
)

定义模型


In [14]:
INPUT_SIZE = 5
SEQUENCE_LENGTH = 7
NUM_LAYERS = 2
HIDDEN_SIZE = 32
OUTPUT_SIZE = 10


class SimpleGRU(nn.Module):
    def __init__(self,
                 input_size=INPUT_SIZE,
                 hidden_size=HIDDEN_SIZE,
                 num_layers=NUM_LAYERS,
                 output_size=OUTPUT_SIZE,
                 sequence_length=SEQUENCE_LENGTH):
        super(SimpleGRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.gru = nn.GRU(input_size, hidden_size,
                          num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size * sequence_length, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)

        out, _ = self.gru(x, h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc1(out)
        return out

训练模型


In [16]:
temp = SimpleGRU()
print(temp)

SimpleGRU(
  (gru): GRU(5, 32, num_layers=2, batch_first=True)
  (fc1): Linear(in_features=224, out_features=10, bias=True)
)


In [18]:
optimizer = torch.optim.Adam(temp.parameters(), lr=0.01, weight_decay=1e-5)
TRAIN_WITH_PROGRESS_BAR(
    MODEL=temp,
    NUM_EPOCHS=1,
    OPTIMIZER=optimizer,
    TRAIN_LOADER=train_loader,
    TEST_LOADER=test_loader,
)

PyTorch Version: 1.11.0
Training on cuda:0
====================================Start training====================================


Epoch 1/1:   0%|          | 0/384 [00:00<?, ?batch/s]


RuntimeError: For unbatched 2-D input, hx should also be 2-D but got 3-D tensor